In [2]:
import numpy as np
import sys
import os
import tensorflow as tf
from keras import optimizers
from keras.layers import Input, Add, concatenate
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.layers import Dense, Flatten, Reshape, Dropout
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from keras.layers import Lambda
from keras.utils import np_utils
from keras.models import load_model
import h5py
from matplotlib.pyplot import imshow
import cv2
import glob
import math
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions



/Users/malavikabindhi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


PATH TO DATA:

In [2]:
#testing the data processing
pcd_path = "/Volumes/Hard Drive/data_object_velodyne/training/velodyne/"
label_path = "/Volumes/Hard Drive/training/label_2/"
calib_path = "/Volumes/Hard Drive/data_object_calib/training/calib/"
img_path = "/Volumes/Hard Drive/data_object_image_2/training/image_2/"

Generate Training Data for PointNet:
(Data generated, processed, and stored in 'train_points.npy' and 'train_labels.npy')

In [86]:
#loading files and paths
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
sys.path.append(BASE_DIR)

def load_pc_from_bin(bin_path):
    """Load a bin file to a np array"""
    num_select = 2048
    selected_points =[]
    obj = np.fromfile(bin_path, dtype = np.float32).reshape(-1,4)
    pc = filter_camera_angle(obj)
    index = np.random.choice(pc.shape[0], num_select, replace=False)
    for i in range(len(index)):
        selected_points.append(pc[index[i]][0:3])
    selected_points = np.array(selected_points).reshape(1,-1,3)# return N*3 array
    return selected_points



def read_calib_file(calib_path):
    """Read a calibration file."""
    data = {}
    with open(calib_path, 'r') as f:
        for line in f.readlines():
            if not line or line == "\n":
                continue
            key, value = line.split(':', 1)
            try:
                data[key] = np.array([float(x) for x in value.split()])
            except ValueError:
                pass
    return data


def proj_to_velo(calib_data):
    """Projection matrix to 3D axis for 3D Label"""
    rect = calib_data["R0_rect"].reshape(3, 3)
    velo_to_cam = calib_data["Tr_velo_to_cam"].reshape(3, 4)
    inv_rect = np.linalg.inv(rect)
    inv_velo_to_cam = np.linalg.pinv(velo_to_cam[:, :3])
    return np.dot(inv_velo_to_cam, inv_rect)


def read_label_from_txt(label_path):
    """Read label from txt file."""
    text = np.fromfile(label_path)
    bounding_box = []
    with open(label_path, "r") as f:
        labels = f.read().split("\n")
        for label in labels:
            if not label:
                continue
            label = label.split(" ")
            if (label[0] == "DontCare"):
                continue
            y_class = [int(label[0]=="Car"), int(label[0]=="Van"), int(label[0]=="Pedestrian")]
            if label[0] == ("Car") or label[0]=="Van" or label[0]=="Pedestrian": #  or "Truck"
                y_labels = list(np.hstack((label[8:15], y_class)))
                bounding_box.append(y_labels)
                break

    if bounding_box:
        data = np.array(bounding_box, dtype=np.float32)
        return data[:, 3:6], data[:, :3], data[:, 6], data[:,7:] 
    else:
        return None, None, None, None

def read_labels(label_path, label_type, calib_path=None, is_velo_cam=False, proj_velo=None):
    """Read labels from txt file.
    Original Label value is shifted about 0.27m from object center.
    So need to revise the position of objects.
    """
    if label_type == "txt": #TODO
        places, size, rotates, y_class = read_label_from_txt(label_path)
        if places is None:
            return None, None, None, None
        rotates = np.pi / 2 - rotates
        dummy = np.zeros_like(places)
        dummy = places.copy()
        if calib_path:
            places = np.dot(dummy, proj_velo.transpose())[:, :3]
        else:
            places = dummy
        if is_velo_cam:
            places[:, 0] += 0.27

    data_combined = []
    for p, r, s , cl in zip(places, rotates, size, y_class):
        ps = np.hstack((cl[:],p[:],s[:]))
        data_combined.append(list(np.append(ps, r)))
        
    return places, rotates, size, y_class

def get_boxcorners(places, rotates, size):
    """Create 8 corners of bounding box from bottom center."""
    corners = []
    for place, rotate, sz in zip(places, rotates, size):
        x, y, z = place
        h, w, l = sz
        if l > 10:
            continue

        corner = np.array([
            [x - l / 2., y - w / 2., z],
            [x + l / 2., y - w / 2., z],
            [x - l / 2., y + w / 2., z],
            [x - l / 2., y - w / 2., z + h],
            [x - l / 2., y + w / 2., z + h],
            [x + l / 2., y + w / 2., z],
            [x + l / 2., y - w / 2., z + h],
            [x + l / 2., y + w / 2., z + h],
        ])

        corner -= np.array([x, y, z])

        rotate_matrix = np.array([
            [np.cos(rotate), -np.sin(rotate), 0],
            [np.sin(rotate), np.cos(rotate), 0],
            [0, 0, 1]
        ])

        a = np.dot(corner, rotate_matrix.transpose())
        a += np.array([x, y, z])
        corners.append(a)
    return np.array(corners)

def filter_camera_angle(places):
    """Filter camera angles for KiTTI Datasets"""
    bool_in = np.logical_and((places[:, 1] < places[:, 0] - 0.27), (-places[:, 1] < places[:, 0] - 0.27))
    # bool_in = np.logical_and((places[:, 1] < places[:, 0]), (-places[:, 1] < places[:, 0]))
    return places[bool_in]

def center_to_sphere(places, size, resolution=0.50, min_value=np.array([0., -50., -4.5]), scale=4, x=(0, 90), y=(-50, 50), z=(-4.5, 5.5)):
    """Convert object label to Training label for objectness loss"""
    x_logical = np.logical_and((places[:, 0] < x[1]), (places[:, 0] >= x[0]))
    y_logical = np.logical_and((places[:, 1] < y[1]), (places[:, 1] >= y[0]))
    z_logical = np.logical_and((places[:, 2] < z[1]), (places[:, 2] >= z[0]))
    xyz_logical = np.logical_and(x_logical, np.logical_and(y_logical, z_logical))
    center = places.copy()
    center[:, 2] = center[:, 2] + size[:, 0] / 2.
    sphere_center = ((center[xyz_logical] - min_value) / (resolution * scale)).astype(np.int32)
    return sphere_center

def sphere_to_center(p_sphere, resolution=0.5, scale=4, min_value=np.array([0., -50., -4.5])):
    """from sphere center to label center"""
    center = p_sphere * (resolution*scale) + min_value
    return center

#derived from yukitsuji/3D_CNN_tensorflow
def process(velodyne_path, label_path=None, calib_path=None, dataformat="bin", label_type="txt", is_velo_cam=False):
    p = []
    pc = None
    bounding_boxes = None
    places = None
    rotates = None
    size = None
    proj_velo = None
    
    filenames_velo = [d for d in sorted(os.listdir(velodyne_path)) if d[0]!='.']
    train_points = None
    train_labels = None
    train_classes = None
    for d in filenames_velo:
        value = d[0:6]
        print(value)
        velo_path = velodyne_path + value + '.bin'
        cal_path = calib_path + value + '.txt'
        lab_path = label_path + value + '.txt'
        cur_points = load_pc_from_bin(velo_path)
        cur_calib = read_calib_file(cal_path)
        proj_velo = proj_to_velo(cur_calib)[:, :3]
        places, rotates, size, classes= read_labels(lab_path, label_type, cal_path , is_velo_cam=is_velo_cam, proj_velo=proj_velo)
        corners = get_boxcorners(places, rotates, size)
        if places is None:
            continue
        if train_points is None:
            train_labels = corners
            train_points = cur_points
            train_classes = classes
        else:
            train_points = np.concatenate((train_points, cur_points), axis = 0)
            train_labels = np.concatenate((train_labels, corners), axis = 0)
            train_classes = np.concatenate((train_classes, classes), axis =0)
            
    return train_points, train_labels, train_classes

In [87]:
train_points, train_labels, train_classes = process(pcd_path, label_path, calib_path=calib_path, dataformat="bin", is_velo_cam=True)

002000
002001
002002
002003
002004
002005
002006
002007
002008
002009
002010
002011
002012
002013
002014
002015
002016
002017
002018
002019
002020
002021
002022
002023
002024
002025
002026
002027
002028
002029
002030
002031
002032
002033
002034
002035
002036
002037
002038
002039
002040
002041
002042
002043
002044
002045
002046
002047
002048
002049
002050
002051
002052
002053
002054
002055
002056
002057
002058
002059
002060
002061
002062
002063
002064
002065
002066
002067
002068
002069
002070
002071
002072
002073
002074
002075
002076
002077
002078
002079
002080
002081
002082
002083
002084
002085
002086
002087
002088
002089
002090
002091
002092
002093
002094
002095
002096
002097
002098
002099
002100
002101
002102
002103
002104
002105
002106
002107
002108
002109
002110
002111
002112
002113
002114
002115
002116
002117
002118
002119
002120
002121
002122
002123
002124
002125
002126
002127
002128
002129
002130
002131
002132
002133
002134
002135
002136
002137
002138
002139
002140
002141
002142

003171
003172
003173
003174
003175
003176
003177
003178
003179
003180
003181
003182
003183
003184
003185
003186
003187
003188
003189
003190
003191
003192
003193
003194
003195
003196
003197
003198
003199
003200
003201
003202
003203
003204
003205
003206
003207
003208
003209
003210
003211
003212
003213
003214
003215
003216
003217
003218
003219
003220
003221
003222
003223
003224
003225
003226
003227
003228
003229
003230
003231
003232
003233
003234
003235
003236
003237
003238
003239
003240
003241
003242
003243
003244
003245
003246
003247
003248
003249
003250
003251
003252
003253
003254
003255
003256
003257
003258
003259
003260
003261
003262
003263
003264
003265
003266
003267
003268
003269
003270
003271
003272
003273
003274
003275
003276
003277
003278
003279
003280
003281
003282
003283
003284
003285
003286
003287
003288
003289
003290
003291
003292
003293
003294
003295
003296
003297
003298
003299
003300
003301
003302
003303
003304
003305
003306
003307
003308
003309
003310
003311
003312
003313

In [91]:
print(train_classes.shape)
np.save('train_classes.npy', train_classes)
np.save('train_points.npy', train_points)
np.save('train_labels.npy', train_labels)

(7481, 3)


In [70]:
print(train_labels.reshape((7481,24)).shape)

(7481, 24)


Generate image data and processing using Resnet-50 model:
(Data generated, processed, and activation layer is stored in 'intermediate_output.npy')

In [5]:
#run this cell only if images need to be processed again
filenames_imgs = [img_path+d for d in sorted(os.listdir(img_path)) if d[0]!='.']
batch_images = None
for d in filenames_imgs:
    print(d)
    original = load_img(d, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch_cur = np.expand_dims(numpy_image, axis=0)
    if batch_images is None:
        batch_images = image_batch_cur
    else:
        batch_images = np.concatenate((batch_images,image_batch_cur ), axis = 0)



/Volumes/Hard Drive/data_object_image_2/training/image_2/000000.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000001.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000002.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000003.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000004.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000005.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000006.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000007.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000008.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000009.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000010.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000011.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000012.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000013.png
/Volumes/Hard Drive/data_object_image_2/training

/Volumes/Hard Drive/data_object_image_2/training/image_2/000121.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000122.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000123.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000124.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000125.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000126.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000127.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000128.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000129.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000130.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000131.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000132.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000133.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000134.png
/Volumes/Hard Drive/data_object_image_2/training

/Volumes/Hard Drive/data_object_image_2/training/image_2/000243.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000244.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000245.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000246.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000247.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000248.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000249.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000250.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000251.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000252.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000253.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000254.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000255.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000256.png
/Volumes/Hard Drive/data_object_image_2/training

/Volumes/Hard Drive/data_object_image_2/training/image_2/000364.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000365.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000366.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000367.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000368.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000369.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000370.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000371.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000372.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000373.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000374.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000375.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000376.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000377.png
/Volumes/Hard Drive/data_object_image_2/training

/Volumes/Hard Drive/data_object_image_2/training/image_2/000485.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000486.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000487.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000488.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000489.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000490.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000491.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000492.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000493.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000494.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000495.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000496.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000497.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/000498.png
/Volumes/Hard Drive/data_object_image_2/training

...

/Volumes/Hard Drive/data_object_image_2/training/image_2/005930.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005931.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005932.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005933.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005934.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005935.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005936.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005937.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005938.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005939.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005940.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005941.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005942.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/005943.png
/Volumes/Hard Drive/data_object_image_2/training

/Volumes/Hard Drive/data_object_image_2/training/image_2/006051.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006052.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006053.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006054.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006055.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006056.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006057.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006058.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006059.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006060.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006061.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006062.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006063.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006064.png
/Volumes/Hard Drive/data_object_image_2/training

/Volumes/Hard Drive/data_object_image_2/training/image_2/006172.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006173.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006174.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006175.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006176.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006177.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006178.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006179.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006180.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006181.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006182.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006183.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006184.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006185.png
/Volumes/Hard Drive/data_object_image_2/training

/Volumes/Hard Drive/data_object_image_2/training/image_2/006293.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006294.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006295.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006296.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006297.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006298.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006299.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006300.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006301.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006302.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006303.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006304.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006305.png
/Volumes/Hard Drive/data_object_image_2/training/image_2/006306.png
/Volumes/Hard Drive/data_object_image_2/training

KeyboardInterrupt: 

In [3]:
from keras.applications import resnet50
resnet_model = resnet50.ResNet50(weights='imagenet')

In [15]:
#run this cell only if images need to be processed again. Otherwise, use the saved file in next cell
processed_image = resnet50.preprocess_input(batch_images.copy())

In [ ]:
#run this cell only if images need to be processed again by resnet. Otherwise, use the saved file in next cell
processed_images = np.load('processed_images.npy')

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#extract final layer called flatten_1 with size 1 by 2048
layer_name = 'flatten_1'
layer = resnet_model.get_layer(layer_name)

#layer = resnet_model.layers[layer_num]
intermediate_layer_model = Model(inputs=resnet_model.input,
                                 outputs=layer.output)
intermediate_output = intermediate_layer_model.predict(processed_images)
print("Shape of Processed Images: ", processed_images.shape)
print("Layer used: ", layer.name)
print("Shape of layer output:", intermediate_output.shape)

np.save('intermediate_output.npy', intermediate_output)


In [3]:
intermediate_output = np.load('intermediate_output.npy')
print(intermediate_output.shape)

(7481, 1, 2048)
